In [11]:
import torch 
import torch.nn as nn 
import torch.optim as optim

import torchvision 
from torch.utils.data import DataLoader,Dataset,TensorDataset
import torch.utils.data as utils
from torchvision import transforms 
import pandas as pd

In [12]:
import matplotlib.image as mpimg
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset

In [13]:
data_dir = "aerial-cactus-identification/"
train_dir = data_dir + "train/"
test_dir = data_dir + "test"

In [14]:
labels = pd.read_csv("aerial-cactus-identification/train.csv")
labels.head()

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1


In [15]:
labels

,id,has_cactus
0,0004be2cfeaba1c0361d39e2b000257b.jpg,1
1,000c8a36845c0208e833c79c1bffedd1.jpg,1
2,000d1e9a533f62e55c289303b072733d.jpg,1
3,0011485b40695e9138e92d0b3fb55128.jpg,1
4,0014d7a11e90b62848904c1418fc8cf2.jpg,1
...,...,...
17495,ffede47a74e47a5930f81c0b6896479e.jpg,0
17496,ffef6382a50d23251d4bc05519c91037.jpg,1
17497,fff059ecc91b30be5745e8b81111dc7b.jpg,1
17498,fff43acb3b7a23edcc4ae937be2b7522.jpg,0


In [16]:
import os 

class ImageData(Dataset):
    def __init__(self,df,data_dir,transform):
        super().__init__()
        
        self.df = df
        self.transform = transform
        self.data_dir = data_dir
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        img_name = self.df.id[index]
        label = self.df.has_cactus[index]
        
        img_path = os.path.join(self.data_dir,img_name)
        image = mpimg.imread(img_path)
        image = self.transform(image)
        return image,label
        
    

In [17]:
data_trans = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.ToTensor()
])

train_data = ImageData(df = labels,data_dir=train_dir,transform=data_trans)



In [18]:
batch_size = 2
valid_size = 0.2
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))

train_idx,valid_idx = indices[split:],indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

trainloader = DataLoader(dataset = train_data,batch_size = 2,sampler = train_sampler)
validloader = DataLoader(dataset = train_data,batch_size = 2,sampler = valid_sampler)



In [19]:
!pip install efficientnet_pytorch


In [20]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b1')


In [21]:
for param in model.parameters():
    param.requires_grad = True

In [22]:
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs,1)

optimizer = optim.Adam(model.parameters())
loss_func = nn.BCELoss()

In [ ]:
loss_log = []
valid_loss = []
valid_loss = np.Inf


for epoch in range(5):
    model.train()
    for ii ,(data,target) in enumerate(trainloader):
        target = target.float()
        
        optimizer.zero_grad()
        output = model(data)
        
        m = nn.Sigmoid()
        loss = loss_func(m(output),target)
        loss.backward()
        
        optimizer.step()
        
        if ii%1000 == 0:
            loss_log.append(loss.item())
            
            
    model.eval()
    for ii,(data,target) in enumerate(validloader):
        target = target.float()
        output = model(data)
        
        m = nn.Sigmoid()
        v_loss = loss_func(m(output),target)
        if ii%1000 == 0:
            valid_loss.append(v_loss.item())
            
            
    print('Epoch: {} - Loss: {:.6f} v_Loss: {:.6f}'.format(epoch + 1, loss.item(),v_loss.item))

In [ ]:
submit = pd.read_csv('../input/sample_submission.csv')
test_data = ImageData(df = submit, data_dir = test_dir, transform = data_transf)
test_loader = DataLoader(dataset = test_data, shuffle=False)

In [ ]:
predict = []
model.eval()
for i, (data, _) in enumerate(test_loader):
    output = model(data)    

    pred = torch.sigmoid(output)
    predicted_vals = pred > 0.5
    predict.append(int(predicted_vals))
    
submit['has_cactus'] = predict
submit.to_csv('submission.csv', index=False)